In [1]:
import os
import xml.etree.ElementTree as ET

In [2]:
classes = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",\
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
print(len(classes))
classes

20


['aeroplane',
 'bicycle',
 'bird',
 'boat',
 'bottle',
 'bus',
 'car',
 'cat',
 'chair',
 'cow',
 'diningtable',
 'dog',
 'horse',
 'motorbike',
 'person',
 'pottedplant',
 'sheep',
 'sofa',
 'train',
 'tvmonitor']

In [3]:
sets=[('2012', 'train'), ('2012', 'val'), ('2007', 'train'), ('2007', 'val'), ('2007', 'test')]

In [4]:
def convert(image_size, box):
    dw = 1./(image_size[0])
    dh = 1./(image_size[1])
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    '''
    官方提供的是
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    不明白为什么要减1
    '''
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x,y,w,h)

In [5]:
def convert_annotation(year, image_id):
    in_file = open('../data/VOCdevkit/VOC%s/Annotations/%s.xml'%(year, image_id))
    out_file = open('labels/VOC%s/%s.txt'%(year, image_id), 'w')
    
    tree=ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
    
    for obj in root.iter('object'): 
        difficult = obj.find('difficult').text # 目标是否难以识别，0为容易，1为困难
        cls = obj.find('name').text
        if cls not in classes or int(difficult) == 1:
            continue

        cls_id = classes.index(cls)
        
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),\
             float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        
        bb = convert((w,h), b)
        
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

In [6]:
for year, image_set in sets:
    if not os.path.exists('labels/VOC%s'%(year)):
        # 添加label目录
        os.makedirs('labels/VOC%s'%(year))
    # 获取训练集、验证集、测试集图片的文件名
    image_ids = open('../data/VOCdevkit/VOC%s/ImageSets/Main/%s.txt'%(year, image_set)).read().strip().split()
    list_file = open('%s_%s.txt'%(year, image_set), 'w')
    for image_id in image_ids:
        list_file.write('../data/VOCdevkit/VOC%s/JPEGImages/%s.jpg\n'%(year, image_id))
        convert_annotation(year, image_id)
    list_file.close()

In [7]:
os.system('cat 2007_train.txt 2007_val.txt 2012_train.txt 2012_val.txt > train.txt')
os.system('cat 2007_test.txt > test.txt')
os.system('rm 20*')

0